---
title: "BfContentItem Implementation Plan"
author: "Content Foundry Team"
summary: "A plan for implementing BfContentItem as a standalone model"
cta: "Review Plan"
---

# BfContentItem Implementation Plan

## Current State

Currently, `BfContentItem` exists only as a props interface and GraphQL type
definition. Content items are stored as properties within the
`BfContentCollection` model rather than as individual database entities. This
makes it difficult to:

1. Query content items directly
2. Establish relationships between content items and other entities
3. Track revisions and changes to individual content items
4. Implement proper access control at the item level

## Implementation Goals

1. Create a proper `BfContentItem` model extending `BfNodeBase`
2. Establish a relationship between collections and items using edges
3. Update GraphQL resolvers to use the new model
4. Update the UI components to work with the new model
5. Migrate existing content items to the new structure

## Technical Design

### 1. Model Definition

Create a new file `packages/bfDb/models/BfContentItem.ts`:

In [1]:
export interface BfContentItemProps extends BfNodeBaseProps {
  title: string;
  body: string;
  slug: string;
  filePath?: string;
  summary?: string;
  author?: string;
  cta?: string;
  href?: string;
}

class BfContentItem extends BfNodeBase<BfContentItemProps> {
  // Implementation details
}

ReferenceError: BfNodeBase is not defined

In [2]:
### 2. Collection to Item Relationship

Create a simple edge relationship between collections and items:

Expected ident at file:///repl.tsx:1:2

  ### 2. Collection to Item Relationship
   ~: Expected ident at file:///repl.tsx:1:2

  ### 2. Collection to Item Relationship
   ~

In [3]:
// In BfContentCollection.ts
async addItem(cv: BfCurrentViewer, item: BfContentItem): Promise<BfEdge> {
  return BfEdge.createBetweenNodes(cv, this, item);
}

async getItems(cv: BfCurrentViewer): Promise<BfContentItem[]> {
  const edges = await BfEdge.queryTargetInstances(
    cv,
    BfContentItem,
    this.metadata.bfGid
  );
  
  return edges;
}

Expected '=>', got '(' at file:///repl.tsx:2:14

  async addItem(cv: BfCurrentViewer, item: BfContentItem): Promise<BfEdge> {
               ~: Expected '=>', got '(' at file:///repl.tsx:2:14

  async addItem(cv: BfCurrentViewer, item: BfContentItem): Promise<BfEdge> {
               ~

With this approach, we're using the standard edge functionality without specifying custom roles. This simplifies the implementation while still maintaining the relationship between collections and items.

### 3. GraphQL Schema Updates

Update GraphQL types to reflect the new model:

In [4]:
// Update resolvers to use the new relationship
export const graphqlBfContentCollectionType = objectType({
  name: "BfContentCollection",
  definition(t) {
    t.implements(graphqlBfNode);
    t.string("title");
    t.string("description");
    t.list.field("items", {
      type: "BfContentItem",
      resolve: async (parent, _args, ctx) => {
        const collection = await ctx.findX(
          BfContentCollection,
          toBfGid(parent.id),
        );

        // Use the new relationship method
        const items = await collection.getItems(ctx.cv);
        return items.map((item) => item.toGraphql());
      },
    });
  },
});

ReferenceError: objectType is not defined

### 4. Migration Strategy

1. **Implementation Phase**:
   - Create the new model and tests
   - Update GraphQL resolvers
   - Update UI components

2. **Data Migration**:
   - Create a migration script to convert existing content
   - For each collection, create item nodes and establish edges
   - Validate the migration with tests

3. **Rollout**:
   - Update the content scanning logic to create proper nodes
   - Switch to the new implementation in the GraphQL layer
   - Update the UI to use the new structure

## Implementation Plan

### Phase 1: Core Model (Week 1)

- [ ] Create BfContentItem model
- [ ] Implement basic operations (create, find, save)
- [ ] Write tests for the model
- [ ] Update BfContentCollection to use relationships

### Phase 2: GraphQL Layer (Week 1-2)

- [ ] Update GraphQL types and resolvers
- [ ] Implement mutations for content items if needed
- [ ] Test GraphQL operations

### Phase 3: UI Components (Week 2)

- [ ] Update ContentItem.tsx component
- [ ] Update ContentCollection.tsx component
- [ ] Test UI with the new model

### Phase 4: Migration (Week 3)

- [ ] Create migration script
- [ ] Test migration with sample data
- [ ] Run migration on development data
- [ ] Validate the migration results

## Technical Challenges

1. **Backward Compatibility**: Ensure the API remains compatible during
   transition
2. **Performance**: Monitor query performance after switching to edge-based
   relationships
3. **Data Integrity**: Ensure no content is lost during migration

## Success Criteria

1. All content items are stored as individual nodes
2. Collections reference items via edges
3. UI functions correctly with the new model
4. No data loss during migration
5. Performance is equal or better than the previous implementation

## Next Steps

1. Set up detailed tasks in the backlog
2. Assign implementation owners
3. Schedule migration windows
4. Create a rollback plan if needed

In [5]:
import {
  BfCurrentViewer,
} from "packages/bfDb/classes/BfCurrentViewer.ts";
const cv = BfCurrentViewer.__DANGEROUS_USE_IN_SCRIPTS_ONLY__createLoggedIn({}, "jupyter", "jupyter")

↱ /home/runner/workspace/packages/bfDb/classes/BfCurrentViewer.ts:195
WARN: Creating Logged in user: undefined
